In [197]:
import requests
import pandas as pd
import numpy as np
import csv
from bs4 import BeautifulSoup

r = requests.get('https://www.worlddata.info/cost-of-living.php')

data = BeautifulSoup(r.text, 'html.parser')
# world_data = [tag for tag in data.find_all(['tr'])]
world_data = []

        
print(world_data)

[]


**data from:** 

***https://visaguide.world/visa-free-countries/salvadoran-passport/***

***https://www.worlddata.info/cost-of-living.php***




In [182]:
for i in world_data:
    #  print(world_data[i].findAll(['td'])[1].text)
    print(world_data[i])

TypeError: list indices must be integers or slices, not str

In [92]:
world_data[0]

[]